# Geneva House Renting Consultancy
Capstone Project, Applied Data Science Capstone by IBM/Coursera, Simone Lisi.

In this notebook, we will collect the top venues within a 500m radius with respct to each prperty we prevously scraped from the immoscout24.ch listing website.

We will then categorize each venue according to the macro-categories: 'Food', 'Shop & Service', 'Travel & Transport', 'Outdoors & Recreation', 'Arts & Entertainment', 'Nightlife Spot', 'Professional & Other Places', 'Residence'.

We will add a feature in our properties database counting the number of nearby venues in each of these macro-categories.


The reults will be stored in the "geneva_allapp.csv", that will be later used for classification.

Some cells are enclosed markdown cells, starting and ending with: ////////////////////
//////////////////// ////////////////////

These cells are set as markdown, as they are not supposed to be run except when we want to scrape new data, or install missing packages. For this they can be set to 'code'. Otherwise, the program will load data previousely scraped and stored.

## Installing packages. Set this cell to 'code' if needed.
!conda install -c anaconda lxml --yes

!conda install -c conda-forge geopy --yes 

!conda install -c conda-forge folium=0.5.0 --yes 


In [344]:
### importing libraries
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
import geopy.distance
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as pl
import numpy as np
%matplotlib inline




In [345]:
### Geneva coordinates
address = 'Geneva, switzerland'

geolocator = Nominatim(user_agent= "To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Geneva are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Geneva are 46.2017559, 6.1466014.


In [346]:
#hidden foursquare credentials
CLIENT_ID = 'xxxx'#  Foursquare ID
CLIENT_SECRET = 'xxxx' #  Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT  = 100

In [347]:
### define function to colect venues around a location

def getVenues(lat, lng, radius=500):
    
    venues_list=[]
  
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
    # make the GET request
    try: 
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])
    
    except:
         # return only relevant information for each nearby venue
        venues_list.append([(
            lat, 
            lng, 
            'missing', 
            'missing', 
            'missing', 
            'missing', 
            'missing' )])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [ 'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category',
                'categoryId']
    
    
        
    return(nearby_venues)

In [348]:
#Loadind dataframe as saved in ScrapingImmoscout24ch_FA.ipynb
df = pd.read_json (r'db_geneva_app_FA.json')
df.head()


,cityName,extraPrice,price,latitude,longitude,numberOfRooms,street,surfaceLiving,zip,pricePerRoom,pricePerSqm,price_dev
6065071,Genève,350.0,2550,46.19363,6.16698,4.0,Rue Le Corbusier 18,85,1208,637.500000,30.000000,-0.171515
6046525,Genève,200.0,4200,46.19646,6.16090,5.5,Route de Malagnou 40E,110,1208,763.636364,38.181818,0.105094
6063101,Genève,170.0,2810,46.20429,6.15891,4.0,Rue du 31-Décembre 16,60,1207,702.500000,46.833333,0.211344
6054808,Genève,250.0,4750,46.19523,6.15391,7.0,Rue Marignac,165,1206,678.571429,28.787879,-0.148491
6037279,Le Lignon,200.0,2700,46.20313,6.09613,6.0,Avenue du Lignon,107,1219,450.000000,25.233645,-0.357799


In [349]:
adress_CTS = 'gare cornavin, Geneva'
location = geolocator.geocode(adress_CTS)
latitude_CTS = location.latitude
longitude_CTS = location.longitude
print('The geograpical coordinate of Cornavin Train Station are {}, {}.'.format(latitude_CTS, longitude_CTS))

The geograpical coordinate of Cornavin Train Station are 46.2098507, 6.141525.


In [350]:
### create a map
address = 'Geneva, Switzerland'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Geneva are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Geneva are 46.2017559, 6.1466014.


In [351]:
map_geneva = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, price, n_rooms, area in zip(df['latitude'], df['longitude'], df['price'], df['numberOfRooms'], df['surfaceLiving']):
    label = '{:.0f}, {:.0f}, {:.0f}'.format(price, n_rooms, area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_geneva)  

    
map_geneva

## //////////////////// //////////////////// ////////////////////
## The following cells are meant to query foursquare. They should be run only once (by converting them into 'code'). Then the data are stored in a dictionary for later use. When using this code from the second time on, skip from here...


venues_per_app_dic = {}

for app_id, lat, lng in zip(df.index, df['latitude'], df['longitude']):

    venues_per_app_dic[app_id] = getVenues(lat, lng)

#len(venues_per_app_dic.keys())
venues_per_app_dic[6059594]

##some calls might have failed. I identify them and then run again the foursquare query for those (in the cell below)
failed_call_id =[]
for key in venues_per_app_dic.keys():
    if venues_per_app_dic[key].iloc[0,3] == 'missing':
        failed_call_id.append(key)
        
len(failed_call_id)   

for element in failed_call_id:
    venues_per_app_dic[element] = getVenues(df.loc[element, 'latitude'], df.loc[element, 'longitude'])

## ... to here
## //////////////////// //////////////////// ////////////////////

In [352]:
#### save/import dictionary of dataframes.
### each key of the dictionary corresponds to a property identifier. The associate dataframe contains 
### informations the top venues around each property

import csv
import os
root = 'venues_per_app'
os.path.join(root, 'some_file.csv')

def dic_save(dic):
    for key, val in dic.items():
        val.to_csv(os.path.join(root, 'data_{}.csv'.format(str(key))))
        
    with open(os.path.join(root, 'key.txt'), "w") as f: #saving keys to file
        f.write(str(list(dic.keys())))

def dic_load():
    """Reading data from keys"""
    with open(os.path.join(root, 'key.txt'), "r") as f:
        keys = eval(f.read())

    dic = {}    
    for key in keys:
        dic[key] = pd.read_csv(os.path.join(root, 'data_{}.csv'.format(str(key))))

    return dic
#### uncomment the line below for saving the dictionary. Leave it commented
#### for  importing it
#dic_save(venues_per_app_dic)

dic2 = dic_load()

In [353]:
## let's check how one of the dataframes looks like
dic2[list(dic2.keys())[3]].head()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,categoryId
0,0,46.19523,6.15391,Le Sesflo,46.196000,6.155568,Italian Restaurant,4bf58dd8d48988d110941735
1,1,46.19523,6.15391,Le Pain Quotidien,46.193110,6.154550,Bakery,4bf58dd8d48988d16a941735
2,2,46.19523,6.15391,Gaucho Churrascaria,46.196381,6.152834,Argentinian Restaurant,4bf58dd8d48988d107941735
3,3,46.19523,6.15391,Tea-room de l'Athénée,46.194739,6.154934,Tea Room,4bf58dd8d48988d1dc931735
4,4,46.19523,6.15391,Osteria Zaza,46.192437,6.153315,Restaurant,4bf58dd8d48988d1c4941735


In [354]:
# removing additional columns Unnamed:..
for key in dic2.keys():
    if('Unnamed: 0.1' in dic2[key].columns):
        dic2[key].drop(axis=1, labels = 'Unnamed: 0.1', inplace = True)
    if('Unnamed: 0' in dic2[key].columns):
        dic2[key].drop(axis=1, labels = 'Unnamed: 0', inplace = True)
        

In [355]:
dic2[list(dic2.keys())[3]].head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,categoryId
0,46.19523,6.15391,Le Sesflo,46.196000,6.155568,Italian Restaurant,4bf58dd8d48988d110941735
1,46.19523,6.15391,Le Pain Quotidien,46.193110,6.154550,Bakery,4bf58dd8d48988d16a941735
2,46.19523,6.15391,Gaucho Churrascaria,46.196381,6.152834,Argentinian Restaurant,4bf58dd8d48988d107941735
3,46.19523,6.15391,Tea-room de l'Athénée,46.194739,6.154934,Tea Room,4bf58dd8d48988d1dc931735
4,46.19523,6.15391,Osteria Zaza,46.192437,6.153315,Restaurant,4bf58dd8d48988d1c4941735


### We will now add all the categories to which a location belongs to each dictionary.
The maximum parents category on foursquare is 4, so we will add 4 new columns.

In [356]:
## loading the dictionary containing, for each subcategory, all the parent categories, as prepared in
## Scraping_foursquare_cat_FA.ipynb
with open('dic_all_cat.json', "r") as f:
        dic_all_cat = eval(f.read())


In [357]:
for key in dic2.keys():
    venues_ids = list(dic2[key].loc[:, 'categoryId' ])
    lv0_list = []
    for element in venues_ids:
        try:
            lv0_list.append(dic_all_cat[dic_all_cat[element][2+0]][0])
        except:
             lv0_list.append(None)
    dic2[key]['Lv0'] = pd.Series(lv0_list)
    
    lv1_list = []
    for element in venues_ids:
        try:
            lv1_list.append(dic_all_cat[dic_all_cat[element][2+1]][0])
        except:
             lv1_list.append(None)
    dic2[key]['Lv1'] = pd.Series(lv1_list)
    
    
    lv2_list = []
    for element in venues_ids:
        try:
            lv2_list.append(dic_all_cat[dic_all_cat[element][2+2]][0])
        except:
             lv2_list.append(None)
    dic2[key]['Lv2'] = pd.Series(lv2_list)
    
    lv3_list = []
    for element in venues_ids:
        try:
            lv3_list.append(dic_all_cat[dic_all_cat[element][2+3]][0])
        except:
             lv3_list.append(None)
    dic2[key]['Lv3'] = pd.Series(lv3_list)
    
    
   


In [358]:
# we look at one of the results as an example
dic2[list(dic2.keys())[7]]

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,categoryId,Lv0,Lv1,Lv2,Lv3
0,46.20976,6.14364,Läderach Chocolatier,46.209567,6.143011,Chocolate Shop,52f2ab2ebcbc57f1066b8b31,Shop & Service,Chocolate Shop,None,None
1,46.20976,6.14364,Parfums De Beyrouth,46.209964,6.145777,Falafel Restaurant,4bf58dd8d48988d10b941735,Food,Falafel Restaurant,None,None
2,46.20976,6.14364,Sushi Boky,46.211446,6.144474,Thai Restaurant,4bf58dd8d48988d149941735,Food,Asian Restaurant,Thai Restaurant,None
3,46.20976,6.14364,Boréal Coffee Shop (Mont Blanc),46.208923,6.144016,Coffee Shop,4bf58dd8d48988d1e0931735,Food,Coffee Shop,None,None
4,46.20976,6.14364,El Faro,46.210634,6.145100,Spanish Restaurant,4bf58dd8d48988d150941735,Food,Spanish Restaurant,None,None
5,46.20976,6.14364,Place des Alpes,46.209259,6.146167,Plaza,4bf58dd8d48988d164941735,Outdoors & Recreation,Plaza,None,None
6,46.20976,6.14364,La Petite Reine,46.210336,6.141070,Bar,4bf58dd8d48988d116941735,Nightlife Spot,Bar,None,None
7,46.20976,6.14364,Olé-Olé,46.211170,6.145456,Tapas Restaurant,4bf58dd8d48988d1db931735,Food,Spanish Restaurant,Tapas Restaurant,None
8,46.20976,6.14364,Geneve Tourisme,46.208430,6.145303,Tourist Information Center,4f4530164b9074f6e4fb00ff,Travel & Transport,Tourist Information Center,None,None
9,46.20976,6.14364,Chez ma cousine,46.207604,6.144076,French Restaurant,4bf58dd8d48988d10c941735,Food,French Restaurant,None,None


In [359]:
### Find all Lv0 values
lv0_list = []
for key in dic2:
    temp_list = list(dic2[key]['Lv0'].unique())
    
    for element in temp_list:
        if element not in lv0_list:
            lv0_list.append(element)
    
lv0_list   


['Food',
 'Shop & Service',
 'Travel & Transport',
 'Outdoors & Recreation',
 'Arts & Entertainment',
 'Nightlife Spot',
 'Professional & Other Places',
 'Residence']

In [360]:
### add Lv0 hot encoding keys to df
for element in lv0_list:
    df[element] = df.apply(lambda row: 0, axis = 1) 

In [361]:
## we check the new dataframe
df.head()

,cityName,extraPrice,price,latitude,longitude,numberOfRooms,street,surfaceLiving,zip,pricePerRoom,pricePerSqm,price_dev,Food,Shop & Service,Travel & Transport,Outdoors & Recreation,Arts & Entertainment,Nightlife Spot,Professional & Other Places,Residence
6065071,Genève,350.0,2550,46.19363,6.16698,4.0,Rue Le Corbusier 18,85,1208,637.500000,30.000000,-0.171515,0,0,0,0,0,0,0,0
6046525,Genève,200.0,4200,46.19646,6.16090,5.5,Route de Malagnou 40E,110,1208,763.636364,38.181818,0.105094,0,0,0,0,0,0,0,0
6063101,Genève,170.0,2810,46.20429,6.15891,4.0,Rue du 31-Décembre 16,60,1207,702.500000,46.833333,0.211344,0,0,0,0,0,0,0,0
6054808,Genève,250.0,4750,46.19523,6.15391,7.0,Rue Marignac,165,1206,678.571429,28.787879,-0.148491,0,0,0,0,0,0,0,0
6037279,Le Lignon,200.0,2700,46.20313,6.09613,6.0,Avenue du Lignon,107,1219,450.000000,25.233645,-0.357799,0,0,0,0,0,0,0,0


In [362]:
# one hot encoding.
lv0_onehot = pd.DataFrame(pd.get_dummies(dic2[list(dic2.keys())[0]]['Lv0'], prefix="", prefix_sep="").sum())


In [363]:
##  For each property we count the number of venues for each category ('Food', etc.)

for app_id in df.index:
    lv0_onehot = pd.DataFrame(pd.get_dummies(dic2[app_id]['Lv0'], prefix="", prefix_sep="").sum())
    for one_hot in lv0_list:
        if one_hot in lv0_onehot.index:
             df.loc[app_id, one_hot] = lv0_onehot.loc[one_hot, 0]
            


In [364]:
## we check the final dataframe
df.head()

,cityName,extraPrice,price,latitude,longitude,numberOfRooms,street,surfaceLiving,zip,pricePerRoom,pricePerSqm,price_dev,Food,Shop & Service,Travel & Transport,Outdoors & Recreation,Arts & Entertainment,Nightlife Spot,Professional & Other Places,Residence
6065071,Genève,350.0,2550,46.19363,6.16698,4.0,Rue Le Corbusier 18,85,1208,637.500000,30.000000,-0.171515,7,3,3,0,0,0,0,0
6046525,Genève,200.0,4200,46.19646,6.16090,5.5,Route de Malagnou 40E,110,1208,763.636364,38.181818,0.105094,22,7,3,2,1,1,0,0
6063101,Genève,170.0,2810,46.20429,6.15891,4.0,Rue du 31-Décembre 16,60,1207,702.500000,46.833333,0.211344,29,3,1,2,1,6,0,0
6054808,Genève,250.0,4750,46.19523,6.15391,7.0,Rue Marignac,165,1206,678.571429,28.787879,-0.148491,21,3,3,1,2,1,0,0
6037279,Le Lignon,200.0,2700,46.20313,6.09613,6.0,Avenue du Lignon,107,1219,450.000000,25.233645,-0.357799,3,1,0,1,0,0,0,0


In [365]:
## Finally, we can store the results in a file for later use 
df.to_csv(r'geneva_allapp.csv')